<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#notes-and-questions" data-toc-modified-id="notes-and-questions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>notes and questions</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Setup---Debug" data-toc-modified-id="Setup---Debug-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Setup - Debug</a></span></li><li><span><a href="#Setup---Imports" data-toc-modified-id="Setup---Imports-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Setup - Imports</a></span></li><li><span><a href="#Setup---working-folder-paths" data-toc-modified-id="Setup---working-folder-paths-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Setup - working folder paths</a></span></li><li><span><a href="#Setup---logging" data-toc-modified-id="Setup---logging-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Setup - logging</a></span></li><li><span><a href="#Setup---virtualenv-jupyter-kernel" data-toc-modified-id="Setup---virtualenv-jupyter-kernel-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Setup - virtualenv jupyter kernel</a></span></li><li><span><a href="#Setup---Initialize-Django" data-toc-modified-id="Setup---Initialize-Django-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Setup - Initialize Django</a></span></li><li><span><a href="#Setup---ExportToContext-instance" data-toc-modified-id="Setup---ExportToContext-instance-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Setup - ExportToContext instance</a></span></li><li><span><a href="#Setup---Initialize-LoggingHelper" data-toc-modified-id="Setup---Initialize-LoggingHelper-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>Setup - Initialize LoggingHelper</a></span></li></ul></li><li><span><a href="#set-up-context-entities-and-relations" data-toc-modified-id="set-up-context-entities-and-relations-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>set up context entities and relations</a></span></li><li><span><a href="#load-sourcenet-data-into-context" data-toc-modified-id="load-sourcenet-data-into-context-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>load sourcenet data into context</a></span><ul class="toc-item"><li><span><a href="#Functions" data-toc-modified-id="Functions-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Functions</a></span><ul class="toc-item"><li><span><a href="#function-create_article_entity" data-toc-modified-id="function-create_article_entity-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>function create_article_entity</a></span></li></ul></li><li><span><a href="#Retrieve-Article-instances" data-toc-modified-id="Retrieve-Article-instances-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Retrieve Article instances</a></span></li><li><span><a href="#Testing" data-toc-modified-id="Testing-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Testing</a></span></li><li><span><a href="#Load-data-from-Article-instances" data-toc-modified-id="Load-data-from-Article-instances-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Load data from Article instances</a></span></li></ul></li><li><span><a href="#TODO" data-toc-modified-id="TODO-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>TODO</a></span></li></ul></div>

# notes and questions

- Back to [Table of Contents](#Table-of-Contents)

Notes:

- probably will need a class that is an Article_Data container - reference to Article_Data, and then all the logic to process the entities and relations contained within.

Questions:

- Do we want an Identifier Type separate from Entity and Relation identifiers?  I think we do, so we can specify the entity type(s) a given identifier should be used on.

    - Created abstract one, and created a concrete entity identifier type.  Will create one for relation identifiers if needed.

# Setup

- Back to [Table of Contents](#Table-of-Contents)

In [1]:
me = "sourcenet-to-context"

## Setup - Debug

- Back to [Table of Contents](#Table-of-Contents)

In [2]:
debug_flag = False

## Setup - Imports

- Back to [Table of Contents](#Table-of-Contents)

In [3]:
import datetime
from django.db.models import Avg, Max, Min
from django.utils.text import slugify
import logging
import six

## Setup - working folder paths

- Back to [Table of Contents](#Table-of-Contents)

In [4]:
%pwd

'/home/jonathanmorgan/work/django/research/work/phd_work/analysis'

In [5]:
# current working folder
current_working_folder = "/home/jonathanmorgan/work/django/research/work/phd_work/analysis"
current_datetime = datetime.datetime.now()
current_date_string = current_datetime.strftime( "%Y-%m-%d-%H-%M-%S" )

## Setup - logging

- Back to [Table of Contents](#Table-of-Contents)

configure logging for this notebook's kernel (If you do not run this cell, you'll get the django application's logging configuration.

In [6]:
logging_file_name = "{}/{}-{}.log.txt".format( current_working_folder, me, current_date_string )
logging.basicConfig(
    level = logging.DEBUG,
    format = '%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    filename = logging_file_name,
    filemode = 'w' # set to 'a' if you want to append, rather than overwrite each time.
)
print( "Logging initialized, to {}".format( logging_file_name ) )

Logging initialized, to /home/jonathanmorgan/work/django/research/work/phd_work/analysis/sourcenet-to-context-2019-09-07-21-54-33.log.txt


## Setup - virtualenv jupyter kernel

- Back to [Table of Contents](#Table-of-Contents)

If you are using a virtualenv, make sure that you:

- have installed your virtualenv as a kernel.
- choose the kernel for your virtualenv as the kernel for your notebook (Kernel --> Change kernel).

Since I use a virtualenv, need to get that activated somehow inside this notebook.  One option is to run `../dev/wsgi.py` in this notebook, to configure the python environment manually as if you had activated the `sourcenet` virtualenv.  To do this, you'd make a code cell that contains:

    %run ../dev/wsgi.py
    
This is sketchy, however, because of the changes it makes to your Python environment within the context of whatever your current kernel is.  I'd worry about collisions with the actual Python 3 kernel.  Better, one can install their virtualenv as a separate kernel.  Steps:

- activate your virtualenv:

        workon research

- in your virtualenv, install the package `ipykernel`.

        pip install ipykernel

- use the ipykernel python program to install the current environment as a kernel:

        python -m ipykernel install --user --name <env_name> --display-name "<display_name>"
        
    `sourcenet` example:
    
        python -m ipykernel install --user --name sourcenet --display-name "research (Python 3)"
        
More details: [http://ipython.readthedocs.io/en/stable/install/kernel_install.html](http://ipython.readthedocs.io/en/stable/install/kernel_install.html)

## Setup - Initialize Django

- Back to [Table of Contents](#Table-of-Contents)

First, initialize my dev django project, so I can run code in this notebook that references my django models and can talk to the database using my project's settings.

In [7]:
# init django
django_init_folder = "/home/jonathanmorgan/work/django/research/work/phd_work"
django_init_path = "django_init.py"
if( ( django_init_folder is not None ) and ( django_init_folder != "" ) ):
    
    # add folder to front of path.
    django_init_path = "{}/{}".format( django_init_folder, django_init_path )
    
#-- END check to see if django_init folder. --#

In [8]:
%run $django_init_path

django initialized at 2019-09-07 21:54:38.208858


In [9]:
# context imports
from context.models import Entity
from context.models import Entity_Identifier_Type
from context.models import Entity_Identifier

# context_text imports
from context_text.article_coding.article_coding import ArticleCoder
from context_text.article_coding.article_coding import ArticleCoding
from context_text.article_coding.open_calais_v2.open_calais_v2_article_coder import OpenCalaisV2ArticleCoder
from context_text.collectors.newsbank.newspapers.GRPB import GRPB
from context_text.collectors.newsbank.newspapers.DTNB import DTNB
from context_text.export.to_context_base.export_to_context import ExportToContext
from context_text.models import Article
from context_text.models import Article_Subject
from context_text.models import Newspaper
from context_text.shared.context_text_base import ContextTextBase

## Setup - ExportToContext instance

- Back to [Table of Contents](#Table-of-Contents)

Make instance, set instance variables.

In [10]:
my_exporter = ExportToContext()

# no variables to set, yet...
my_exporter.set_article_uuid_id_type_name( ExportToContext.ENTITY_ID_TYPE_ARTICLE_NEWSBANK_ID )

## Setup - Initialize LoggingHelper

- Back to [Table of Contents](#Table-of-Contents)

Create a LoggingHelper instance to use to log debug and also print at the same time.

Preconditions: Must be run after Django is initialized, since `python_utilities` is in the django path.

In [11]:
# python_utilities
from python_utilities.logging.logging_helper import LoggingHelper

# init
my_logging_helper = LoggingHelper()
my_logging_helper.set_logger_name( me )
log_message = None

# set up context entities and relations

- Back to [Table of Contents](#Table-of-Contents)

Set up the following entities and relations in context:

Entities:

- person - a person!
- article - Article, used to detect reporters, subjects, and sources.
- organization - Any type of organized group (of organizations, of people, etc.).
- newspaper - A newspaper, which can be used to cluster articles, employees, sources, etc.

Relations:

- from newspaper

    - newspaper_reporter - Reporter at a newspaper, evidence of which is byline on an article in that newspaper.  FROM newspaper TO person (reporter) THROUGH article.
    - newspaper_source - Person quoted in an article published by a newspaper. FROM newspaper TO person (source) THROUGH article.
    - newspaper_subject - Subject of an article published in a given newspaper.  FROM newspaper TO person (subject, including sources) THROUGH article.
    - newspaper_article - Article published in a particular newspaper.  FROM newspaper TO article.

- from article

    - author - Author/Reporter of an article - FROM article TO reporter.
    - subject - Subject of a story.  FROM article TO subject person.
    - source - Source quoted in an article - FROM article TO source person.

- through article

    - article_container - Parent for relations based on entities being mentioned in the same article.  To start, just people, but eventually, for example, could also include location.
    - mentioned - Mentioned in an article.  FROM reporter/author TO subject THROUGH article.
    - quoted - The \"from\" person quoted the \"to\" person in a publication.  FROM reporter TO source THROUGH article.
    - same_article_sources - Sources in the same article, FROM source person TO source person THROUGH article.
    - same_article_subjects - Two people who are in a particular article together (includes subjects and sources).
    - shared_byline - Shared Byline on an article - joint authors - FROM author TO author THROUGH article.

Then, export them to JSON fixture files using manage.py / django-admin dumpdata ( [https://docs.djangoproject.com/en/dev/ref/django-admin/#django-admin-dumpdata](https://docs.djangoproject.com/en/dev/ref/django-admin/#django-admin-dumpdata) ) so they can be imported using python manage.py or django-admin loaddata ( [https://docs.djangoproject.com/en/dev/ref/django-admin/#django-admin-loaddata](https://docs.djangoproject.com/en/dev/ref/django-admin/#django-admin-loaddata) ) rather than having to input them in the admin:

    python manage.py dumpdata [app_label[.ModelName] [app_label[.ModelName] ...]] --indent INDENT --output <output_file_path>
    
    python manage.py dumpdata \
        --indent 4 \
        --output context-sourcenet_entities_and_relations.json \
        context.Entity_Identifier_Type \
        context.Entity_Relation_Type \
        context.Entity_Relation_Type_Trait \
        context.Entity_Type \
        context.Entity_Type_Trait \
        context.Trait_Type \
        context.Term \
        context.Term_Relation \
        context.Term_Relation_Type \
        context.Vocabulary \
        
    No line breaks:
    
        python manage.py dumpdata --indent 4 --output context-sourcenet_entities_and_relations.json context.Entity_Identifier_Type context.Entity_Relation_Type context.Entity_Relation_Type_Trait context.Entity_Type context.Entity_Type_Trait context.Trait_Type context.Term context.Term_Relation context.Term_Relation_Type context.Vocabulary

# load sourcenet data into context

- Back to [Table of Contents](#Table-of-Contents)

preparation:

- make an ID type for sourcenet person ID (ID 1, name `person_sourcenet_id`).
- also make ID type for OpenCalais URI for person (ID 2, name `person_open_calais_uuid`).
- make an ID type for sourcenet article ID (ID 3, name `article_sourcenet_id`).
- make an ID type for newsbank article ID (ID 2, name `article_newsbank_id`).

logic:

- get all Article_Data created by automated coder, open calais coder type.
- for each:

    - Article processing:
    
        - look for article entity in context.  If not created, create one.
        - Store the ID and entity reference for the Article.

    - Author/Reporter processing.
    
        - look for author person entity.
        - create "article" entity 

## Functions

- Back to [Table of Contents](#Table-of-Contents)

### function create_article_entity

- Back to [Table of Contents](#Table-of-Contents)

Now in `ExportToContext` class.

## Retrieve Article instances

- Back to [Table of Contents](#Table-of-Contents)

In [12]:
# look for publications that have article data:
# - coded by automated coder
# - with coder type of "OpenCalais_REST_API_v2"

# get automated coder
automated_coder_user = ArticleCoder.get_automated_coding_user()

log_message = "{} - Loaded automated user: {}, id = {}".format( datetime.datetime.now(), automated_coder_user, automated_coder_user.id )
my_logging_helper.output_message( log_message, do_print_IN = True, log_level_code_IN = logging.INFO )

2019-09-07 21:55:13.492050 - Loaded automated user: automated, id = 2


In [13]:
# find articles with Article_Data created by the automated user...
article_qs = Article.objects.filter( article_data__coder = automated_coder_user )

# ...and specifically coded using OpenCalais V2...
article_qs = article_qs.filter( article_data__coder_type = OpenCalaisV2ArticleCoder.CONFIG_APPLICATION )

# ...and finally, we just want the distinct articles by ID.
article_qs = article_qs.order_by( "id" ).distinct( "id" )

# count?
article_count = article_qs.count()
log_message = "Found {} articles".format( article_count )
my_logging_helper.output_message( log_message, do_print_IN = True, log_level_code_IN = logging.INFO )

Found 43843 articles


## Testing

- Back to [Table of Contents](#Table-of-Contents)

First, make unit tests to test convenience methods added to the following models, in `context/tests/models/`:

- // `Entity_Identifier_Type` - test_Entity_Identifier_Type_model.py
- // `Entity_Identifier` - test_Entity_Identifier_model.py
- // `Entity` - test_Entity_model.py

Also, // move instance creation class methods along with their constants over into "TestHelper" from test_Entity_Identifier_model.py, so they can be re-used across test classes.

To run: `python manage.py test context.tests`

TODO:

- test_entity_model.py

    - test_get_entity_for_identifier

- test_export_to_context.py

    - test_get_article_uuid_id_type
    - test_set_article_uuid_id_type_name

Next, get a single article to work with for now.

In [14]:
test_article_qs = None
test_article = None
test_entity = None

# slice a single article off the QuerySet.
test_article_qs = article_qs[ : 1 ]
test_article = test_article_qs.get()

Make an entity for the article.

In [15]:
my_exporter.create_article_entity( test_article )

## Load data from Article instances

- Back to [Table of Contents](#Table-of-Contents)

To start, we are going to break this work up into pieces, do one thing at a time, building a function for each thing above that accepts an `Article_Data` instance and does its work, that we can then move into an object:

- Add all articles to context as entities of type "article".
- Add all people who are either reporters/authors or subjects to `context` as entities of type "person", with identifier of type `person_sourcenet_id` set to their internal django/database "id", and with identifier of type `person_open_calais_uuid` set to their OpenCalais ID, if they have one.  If they have any other types of IDs, add them too, untyped.
- For all reporters/authors:

    - create a relation of type "`author`" between the article's entity (FROM) and the entity of the person (TO) for each author.
    - if multiple authors, for each pair of authors, create a relation of type "`shared_byline`" between the two (it is undirected), THROUGH the article.

- For all subjects, including sources:

    - create a relation of type "`subject`" between the article's entity (FROM) and the entity of the subject person (TO).
    - if multiple subjects, for each pair of subjects, create a relation of type "`same_article_subjects`" between the two (it is undirected), THROUGH the article.
    - create a relation of type "`mentioned`" between each of the article's authors (FROM) and the subject (TO), THROUGH the article.

- For all sources

    - create a relation of type "`source`" between the article's entity and the entity for the source person.
    - if multiple sources, for each pair of sources, create a relation of type "`same_article_sources`" between the two (it is undirected), THROUGH the article.
    - create a relation of type "`quoted`" between each of the article's authors (FROM) and the source (TO), THROUGH the article.

Convenience methods:

- method to find entity - based on type and identifier (accept all the fields that make sense, including optional identifier type instance).
- method to find relation - based on type, etc.

Notes:

- all ties are undirected.
- relations can have three foreign keys into Entity - FROM, TO, and THROUGH (for a containing relationship, like the article that included two sources that we are relating).

In [15]:
my_exporter.process_articles( article_qs )


Summary:
- good count: 43843
- > 1 count: 0
- 0 count: 0
- unexpected count: 0


# TODO

- Back to [Table of Contents](#Table-of-Contents)

TODO:

- // take the "`create_article_entity()`" function and put it in a class for loading sourcenet articles into  context

    - `context_text/export/to_context_base/export_to_context.py` - class `ExportToContext`

- build unit test class for this loading class, and add one for creating a fake article, then making an entity out of it. Check entity, traits, and identifiers.